In [3]:
import json
import os
import json
from selenium import webdriver
import time
import re
from stravalib.client import Client


def initialize_json(json_path):
    if not os.path.exists(json_path):
        
        print('This is the first time you use this. Please login to https://www.strava.com/settings/api and create a new application. ')
        
        client_id = input(f"Please enter your client id: ")
        client_secret = input(f"Please enter your client secret: ")
        
        client = Client()
        request_scope = ["read_all", "profile:read_all", "activity:read_all"]
        redirect_url = "http://127.0.0.1:5000/authorization"

        # Creates an authorization URL using stravalib
        url = client.authorization_url(
            client_id=client_id,
            redirect_uri=redirect_url,
            scope=request_scope,
        )

        # Opens the URL and automatically gets the necessary code
        driver = webdriver.Firefox()
        driver.get(url)
        match = re.search(r"authorization?state=&code=([^&]+)", driver.current_url)
        while not match:
            time.sleep(0.5)
            match = re.search(r"code=([^&]+)", driver.current_url)
        code = match.group(1)
        driver.close()

        #Exchanges the code for an access token. Token is stored in json-file.
        client = Client()
        token_response = client.exchange_code_for_token(client_id=client_id, client_secret=client_secret, code=code)

        data = {'client_id': client_id,
                'client_secret': client_secret,
                'token': token_response
               }

        with open(json_path, "w") as file:
            json.dump(data, file, indent=4)

def getAuthenticatedClient(json_path = 'pw'):
    # Make sure the authentification is possible
    initialize_json(json_path)
    
    client = Client()
    with open(json_path, "r") as file:
            data = json.load(file)

    # Access values (client_id, client_secret, tokens)
    client_id = data.get("client_id")       
    client_secret = data.get("client_secret")  
    tokens = data.get("token")

    # Immediately refreshes token and stores it. New token is valid for 6 hours.
    token_response = client.refresh_access_token(
        client_id=client_id,
        client_secret=client_secret,
        refresh_token=tokens['refresh_token'])
    
    new_access_token = token_response["access_token"]
    
    data = {'client_id': client_id,
            'client_secret': client_secret,
            'token': token_response}
    
    with open(json_path, "w") as file:
        json.dump(data, file, indent=4)

    # Returns an authenticated client valid for 6 hours.
    return Client(access_token = new_access_token)

In [4]:
#sets up an authenticated client. 
client = getAuthenticatedClient()

No rates present in response headers


In [5]:
#returns an iterator over the activities
client.get_activities()

<BatchedResultsIterator entity=SummaryActivity>

In [6]:
#gets information on the athlete
client.get_athlete()

DetailedAthlete(bound_client=<stravalib.client.Client object at 0x10a539330>, id=67722075, city=None, country=None, created_at=datetime.datetime(2020, 9, 4, 20, 38, 39, tzinfo=TzInfo(UTC)), firstname='Moritz', lastname='Venzin', premium=True, profile='avatar/athlete/large.png', profile_medium='avatar/athlete/medium.png', resource_state=3, sex='M', state=None, summit=True, updated_at=datetime.datetime(2025, 2, 9, 20, 40, 32, tzinfo=TzInfo(UTC)), bikes=[SummaryGear(distance=1102561.0, id='b15312545', name='BMC diablo', primary=False, resource_state=2), SummaryGear(distance=406620.0, id='b15308047', name='deRosa', primary=False, resource_state=2)], clubs=[SummaryClub(bound_client=None, id=1235616, name='Transibérica Ultracycling', resource_state=2, activity_types=[ActivityType(root='Handcycle'), ActivityType(root='EBikeRide'), ActivityType(root='VirtualRide'), ActivityType(root='Velomobile'), ActivityType(root='Ride')], city='Bilbao', country='Spain', cover_photo='https://dgalywyr863hv.cl